# Calculation of Reference Evapotranspiration

@author: Caroline Gasten

The present notebook calculates the reference evapotranspiration using the FAO Penman Monteith equation (https://www.fao.org/3/X0490E/x0490e00.htm)

# Settings

In [ ]:
# load packages
import xarray as xr
import os
import numpy as np
import rasterio
from rasterio.enums import Resampling
import rioxarray
import matplotlib.pyplot as plt

In [ ]:
#input paths
path_input = #path to daily ERA5 data
path_dem = #path to digital elevation model (DEM)

#output path
path_output = #path to store daily PET data

In [ ]:
#specification of spatial subset for DEM
lons = slice(22.5, 51.75)
lats = slice( 15.75, -4.75)

## Required inputs & outputs
|variable|variable name long|input unit|required unit/output unit|
|---|---|---|---|
|Tmax_d|maximum air temperature daily |K|C|
|Tmin_d|minimum air temperature daily |K|C|
|slr|surface net longwave radiation daily |J/(m2*day)|MJ/(m2*day)|
|ssr|surface net shortwave radiation daily |J/(m2*day)|MJ/(m2*day)|
|u_10|10-m wind speed u-component |m/s|m/s|
|v_10|10-m wind speed v-component |m/s|m/s|
|Tdew_2|2-m dewpoint temperature |K|C|
|p_msl|mean sea level pressure |Pa|kPa|
|dem|elevation from DEM |m|m|
|ET_0|reference evapotranspiration|-|mm/day|



## Load DEM

In [ ]:
#load DEM
dem = rioxarray.open_rasterio(os.path.join(path_dem, 'srtm_025deg_HoA.tiff'), mask_and_scale=True).sel(y=lats, x=lons, band=1)

In [ ]:
#check for null-values
dem.isnull().plot()
plt.title("Null values for the DEM")

## Functions to calculate reference evapotranspiration

In [ ]:
#function to calculate reference evapotranspiration
def ref_ET(r_ss, r_sl, tmax, tmin, tdewmax, tdewmin, u10, v10, p_msl, dem):
    # net radiation
    r_n =  r_ss + r_sl #both formulated positive in downward direction
    
    # ground heat flux
    G=0 #for daily calculations assumption that ground heat flux is 0
    
    #daily mean temperature
    tmean = (tmax+tmin)/2
    
    #windspeed
    ws10 = np.sqrt(u10**2 + v10**2)
    ws2 = ws10*(4.87/np.log(67.8*10-5.42))
    
    #saturation and actual vapour pressure
    e_s = (sat_vap_press(tmax) + sat_vap_press(tmin))/2
    e_a = (sat_vap_press(tdewmax) + sat_vap_press(tdewmin))/2
    
    #slope of vapour pressure curve
    Delta = 4098 * sat_vap_press(tmean) / (tmean + 237.3)**2
    
    # surface pressure using barometric formula
    g = 9.80665 #m/s2
    Mo = 0.0289644 #kg/mol 
    R_air = 8.3144621 #J/(mol*K)
    lapse_rate = -0.0065 #K/m

    power = -g * Mo / (R_air * lapse_rate)
    p_fact = ((288.15 + lapse_rate*dem)/288.15)**power
    p_surf = p_msl * p_fact
    
    #psychrometric constant
    gamma = 0.665 * 10**(-3) * p_surf
    
    #calculation of reference evapotranspiration
    num = 0.408 * Delta * (r_n - G) + gamma * 900 /(tmean + 273) * ws2 * (e_s - e_a)
    denom = Delta + gamma * (1 + 0.34 * ws2)
    ET_0 = num/denom
    ET_0 = ET_0.rename('et0') #mm/day
    
    return ET_0

In [ ]:
#function to calculate saturation vapour pressure
def sat_vap_press(T):
    es_T = 0.6108 * np.exp(17.27 * T / (T+237.3))
    return es_T

## Calculate daily reference evapotranspiration and save

In [ ]:
#loop through years and calculate daily reference evapotranspiration based on daily input datasets
for year in range(1993,2023):
    print(year)
    
    #load datasets
    tmax = xr.open_dataarray(os.path.join(path_input, 'era5_t2mmax_%04d_daily.nc' %(year))).sel(y=lats,x=lons) - 273.15 #K to C
    tmin = xr.open_dataarray(os.path.join(path_input, 'era5_t2mmin_%04d_daily.nc' %(year))).sel(y=lats,x=lons) - 273.15 #K to C
    tdewmax = xr.open_dataarray(os.path.join(path_input, 'era5_2dmax_%04d_daily.nc' %(year))).sel(y=lats,x=lons) - 273.15 #K to C
    tdewmin = xr.open_dataarray(os.path.join(path_input, 'era5_2dmin_%04d_daily.nc' %(year))).sel(y=lats,x=lons) - 273.15  #K to C
    u10 = xr.open_dataarray(os.path.join(path_input, 'era5_u10_%04d_daily.nc' %(year))).sel(y=lats,x=lons) # m/s
    v10 = xr.open_dataarray(os.path.join(path_input, 'era5_v10_%04d_daily.nc' %(year))).sel(y=lats,x=lons) # m/s
    p_msl = xr.open_dataarray(os.path.join(path_input, 'era5_msl_%04d_daily.nc' %(year))).sel(y=lats,x=lons) / 1000 # Pa to kPa
    r_ss = xr.open_dataarray(os.path.join(path_input, 'era5_ssr_%04d_daily.nc' %(year))).sel(y=lats,x=lons) * 10**(-6) # J/m2 to MJ/m2
    r_sl = xr.open_dataarray(os.path.join(path_input, 'era5_str_%04d_daily.nc' %(year))).sel(y=lats,x=lons) * 10**(-6) # J/m2 to MJ/m2
    
    #calculate reference evapotranspiration
    ET_0 = ref_ET(r_ss, r_sl, tmax, tmin, tdewmax, tdewmin, u10, v10, p_msl, dem)
    
    ET_0.to_netcdf(os.path.join(path_output, 'PET_%04d_daily.nc' %(year)))           